In [130]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import csv
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import norm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [131]:
#df_train contains data as is
df_train = pd.read_csv('train.csv')

In [132]:
#df_test contains data as is
df_test = pd.read_csv('test.csv')

In [133]:
print(f'Train contains {df_train.shape[1]} columns, {df_train.shape[0]} rows')
print(f'Test contains {df_test.shape[1]} columns, {df_test.shape[0]} rows')

Train contains 81 columns, 1460 rows
Test contains 80 columns, 1459 rows


In [134]:
#удаляем все колонки, в которых более 15% пустых значений - PoolQC, MiscFeatures, Alley, Fence, FirePlaceQu, LotFrontage
df_train = df_train.drop(['PoolQC'], 1)
df_train = df_train.drop(['MiscFeature'], 1)
df_train = df_train.drop(['Alley'], 1)
df_train = df_train.drop(['Fence'], 1)
df_train = df_train.drop(['FireplaceQu'], 1)
df_train = df_train.drop(['LotFrontage'], 1)

#в колонках GarageX одинаковое количество пропущенных значений
#при этом наиболее важная информация по гаражам содержится в колонке GarageCars
#следовательно, остальные колонки GarageX можно удалить
df_train = df_train.drop(['GarageCond'], 1)
df_train = df_train.drop(['GarageType'], 1)
df_train = df_train.drop(['GarageYrBlt'], 1)
df_train = df_train.drop(['GarageFinish'], 1)
df_train = df_train.drop(['GarageQual'], 1)
df_train = df_train.drop(['BsmtExposure'], 1)
df_train = df_train.drop(['BsmtFinType2'], 1)
df_train = df_train.drop(['BsmtFinType1'], 1)

df_train = df_train.drop(['BsmtCond'], 1)
df_train = df_train.drop(['BsmtQual'], 1)
df_train = df_train.drop(['MasVnrArea'], 1)
df_train = df_train.drop(['MasVnrType'], 1)

#Electrical имеет всего один пропуск, поэтому удаляем строку, а не колонку
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

#удаляю переменные с низкой вариативностью
df_train = df_train.drop(['Utilities'],1)
df_train = df_train.drop(['BsmtFinSF2'],1)


#перпеменные, влияние которых не очевидно
df_train = df_train.drop(['Condition1'],1)
df_train = df_train.drop(['Condition2'],1)
df_train = df_train.drop(['RoofStyle'],1)
df_train = df_train.drop(['RoofMatl'],1)
df_train = df_train.drop(['Exterior1st'],1)
df_train = df_train.drop(['Exterior2nd'],1)

# удаляю переменные, имеющие значительную корреляцию с аналогичными по смыслу переменными
df_train = df_train.drop(['TotRmsAbvGrd'],1)
df_train = df_train.drop(['GarageArea'],1)

#удаляем выбросы в цене
df_train = df_train.drop(df_train[(df_train.SalePrice < 200000) & (df_train.GrLivArea > 4000)].index)


In [135]:
print(f'Train contains {df_train.shape[1]} columns, {df_train.shape[0]} rows')
print(f'Test contains {df_test.shape[1]} columns, {df_test.shape[0]} rows')

Train contains 53 columns, 1457 rows
Test contains 80 columns, 1459 rows


## лог-трансформация

In [138]:
def log_transform(df, col):
    zero = df[df[col] == 0].shape[0]
    print(zero)    
    if zero:
        hascol = 'Has'+ col
        df[hascol] = pd.Series(len(df[col]), index=df.index)
        df[hascol] = 0 
        df.loc[df[col]>0, hascol] = 1
        df.loc[df[hascol]==1, col] = np.log(df[col])
    else:
        df[col] = np.log(df[col])

log_transform(df_train, 'GrLivArea')
log_transform(df_train, 'TotalBsmtSF')

0
37


In [ ]:
#MSSubClass
# 10 - low SalePrice median 
# 20 - medium SalePrice median 
# 30 - high SalePrice median 
mssubclass_category_codes = {
    20: 30, 30: 10, 40: 20, 45: 10, 50: 30, 60: 30,
    70: 20, 75: 20, 80: 20, 85: 10, 90: 10, 120: 20,
    150: 20, 160: 20, 180: 10, 190: 20 
}
 
#MSZoning
#only this codes in dataset: RL, RM, FV, RH, C (all)
mszoning_codes = {'C (all)': 10, 'RH': 20, 'RM': 30, 'RL': 40, 'FV': 50}
#Street
street_codes = {'Grvl': 1, 'Pave': 2}

#LotShape
lotshape_codes = {'Reg': 10, 'IR1': 20, 'IR2': 30, 'IR3': 40}
#LandContour
landcontour_codes = {'Bnk': 10, 'Lvl': 20, 'Low': 30, 'HLS': 40}
#Utilities
utilities_codes = {'ELO': 1, 'NoSeWa': 2, 'NoSewr': 3, 'AllPub': 4}
#LotConfig
lotconfig_codes = {'CulDSac': 1, 'Corner': 2, 'FR2': 3, 'FR3': 4, 'Inside': 5}
#LandSlope
landslope_codes = {'Sev': 1, 'Mod': 2, 'Gtl': 3}
#Neighborhood
#inaccuracy in dataset:
# not 'Names' but 'NAmes'
neighborhood_codes_old = {
    'Blmngtn': 1, 'Blueste': 2, 'BrDale': 3, 'BrkSide': 4, 'ClearCr': 5, 'CollgCr': 6,
    'Crawfor': 7, 'Edwards': 8, 'Gilbert': 9, 'IDOTRR': 10, 'MeadowV': 11, 'Mitchel': 12,
    'NAmes': 13, 'NoRidge': 14, 'NPkVill': 15, 'NridgHt': 16, 'NWAmes': 17, 'OldTown': 18,
    'SWISU': 19, 'Sawyer': 20, 'SawyerW': 21, 'Somerst': 22, 'StoneBr': 23, 'Timber': 24, 'Veenker': 25
}

neighborhood_codes = {
    #1
    'Blmngtn': 16,
    'Blueste': 8,
    'BrDale': 3,
    'BrkSide': 6,
    'ClearCr': 18,
    'CollgCr': 17,
    'Crawfor': 19,
    'Edwards': 5, 
    'Gilbert': 14, 
    #10    
    'IDOTRR': 2, 
    'MeadowV': 1, 
    'Mitchel': 12,
    'NAmes': 10, 
    'NoRidge': 24,
    'NPkVill': 11, 
    'NridgHt': 25,
    'NWAmes': 15, 
    'OldTown': 4,
    'SWISU': 9, 
    #20
    'Sawyer': 7,  
    'SawyerW': 13, 
    'Somerst': 21, 
    'StoneBr': 23,
    'Timber': 22, 
    'Veenker': 20
}

#Condition 1 and 2
condition_codes = {
    'Artery': 1, 'Feedr': 2, 'Norm': 3, 'RRNn': 4, 'RRAn': 5,
    'PosN': 6, 'PosA': 7, 'RRNe': 8, 'RRAe': 9
}

#BldgType
#inaccuracy in dataset:
# not 'TwnhsI' but 'Twnhs'
# not '2FmCon' but '2fmCon'
# not 'Duplx' but 'Duplex'
bldgtype_codes = {'TwnhsE': 1, 'Twnhs': 2, '1Fam': 3, '2fmCon': 4, 'Duplex': 5}

#HouseStyle
housestyle_codes = {'1Story': 1, '1.5Unf': 2, '1.5Fin': 3, '2Story': 4, '2.5Unf': 5, '2.5Fin': 6, 'SFoyer': 7, 'SLvl': 8}

#RoofStyle
roofstyle_codes = {'Flat': 1, 'Gable': 2, 'Gambrel': 3, 'Hip': 4, 'Mansard': 5, 'Shed': 6}

#RoofMatl
roofmatl_codes = {'Membran': 1, 'WdShake': 2, 'WdShngl': 3, 'Roll': 4, 'Tar&Grv': 5, 'Metal': 6, 'CompShg': 7, 'ClyTile': 8}

#Exterior 1st and 2nd 
#inaccuracy in dataset:
# neither 'WdShing' not 'Wd Sdng' but 'Wd Shng'
# not 'CemntBd' but 'CmentBd'
# not 'BrkComm' but 'Brk Cmn'
exterior_codes = {
    'WdShing': 1, 'Wd Shng': 1, 'Wd Sdng': 2, 'AsbShng': 3, 'AsphShn': 4, 'CBlock': 5, 'CmentBd': 6, 'CemntBd': 6,
    'HdBoard': 7, 'Stone': 8, 'PreCast': 9, 'Other': 10, 'Plywood': 11, 'BrkComm': 12, 'Brk Cmn': 12,
    'VinylSd': 13, 'MetalSd': 14, 'Stucco': 15, 'ImStucc': 16, 'BrkFace': 17
} 

#MasVnrType
masvnrtype_codes = {'None': 1, 'Stone': 2, 'CBlock': 3, 'BrkCmn': 4, 'BrkFace': 5}

#ExterQual
#ExterCond
#HeatingQC
#KitchenQual
fivelevel_codes = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5} 

#BsmtQual
#BsmtCond
#FireplaceQu
#GarageQual
#GarageCond
sixlevel_codes = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}

#Foundation
foundation_codes = {'Wood': 1, 'BrkTil': 2, 'CBlock': 3, 'Stone': 4, 'Slab': 5, 'PConc': 6} 

#BsmtExposure
bsmtexposure_codes = {'Gd': 1, 'Av': 2, 'Mn': 3, 'No': 4, 'NA': 5}

#BsmtFinType1 and BsmtFinType2
bsmtfintype_codes = {'NA': 1, 'Unf': 2, 'LwQ': 3, 'Rec': 4, 'BLQ': 5, 'ALQ': 6, 'GLQ': 7}

#Heating
heating_codes = {'Wall': 1, 'OthW': 2, 'Floor': 3, 'GasA': 4, 'Grav': 5, 'GasW': 6}

#CentralAir
yno_codes = {'N': 0, 'Y': 1}

#Electrical
electrical_codes = {'FuseP': 1, 'FuseF': 2, 'Mix': 3, 'FuseA': 4, 'SBrkr': 5}

#Functional
functional_codes = {'Sal': 1, 'Sev': 2, 'Maj2': 3, 'Maj1': 4, 'Mod': 5, 'Min2': 6, 'Min1': 7, 'Typ': 8} 

# GarageType
garagetype_codes = {
    'NA': 1, 'CarPort': 2, 'Detchd': 3, 'Attchd': 4,
    'Basment': 5, 'BuiltIn': 6, '2Types': 7
}

#GarageFinish
garagefinish_codes = {'NA': 1, 'Unf': 2, 'RFn': 3, 'Fin': 4}

#PavedDrive
paveddrive_codes = {'N': 1, 'P': 2, 'Y': 3}

#PoolQC
poolqc_codes = {'NA': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5} 

#Fence
fence_codes = {'NA': 1, 'MnWw': 2, 'GdWo': 3, 'MnPrv': 4, 'GdPrv': 5} 

#MiscFeature
miscfeature_codes = {'NA': 0, 'Othr': 1, 'Shed': 1, 'Gar2': 1, 'Elev': 2, 'TenC': 2}

#SaleType
saletype_codes = {
    'WD': 1, 'CWD': 2, 'VWD': 3, 'New': 4, 'COD': 5, 'Con': 6,
    'ConLw': 7, 'ConLI': 8, 'ConLD': 9, 'Oth': 10
} 

#SaleCondition
salecondition_codes = {'Normal': 1, 'Abnorml': 2, 'AdjLand': 3, 'Alloca': 4, 'Family': 5, 'Partial': 6} 
